##### Set up packages and directories

In [99]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
from tqdm.auto import tqdm
from Comparative_Analysis import Sequence_Analysis_Routines as sar
from Comparative_Analysis import Utilities as util
from Comparative_Analysis import Alignment as align
import random
import copy
from joblib import Parallel, delayed
import os
import subprocess
from Bio.Blast import NCBIWWW, NCBIXML

In [100]:
project_dir = 'D:/Project_Data/Project_6'
literature_datasets_dir = project_dir + '/Datasets/Data_From_Publications'
output_dir = project_dir + '/Output'
refseq_dir = 'D:/Tests/NCBI_Dataset_Mycobacteria'
cryptic_output_path = "D:/Project_Data/CRYPTIC_DATA/Cryptic_Data_Analysis"
num_cores = 16
core_numbers = list(range(1, num_cores+1))

In [119]:
species_list = util.list_dirs(refseq_dir)
reference_species = 'GCF_000195955.2'
species_list_excl_ref = [x for x in species_list if x!= reference_species]
len(species_list)

88

In [104]:
min_nts = 60

##### Translation function

In [105]:
codon_dict = {}
with open('D:/Project_Data/Project_3/Datasets/Reference_Tables/Standard_Code.txt') as f:
    for l in f:
        codon_dict[str(l[1:4])] = l[5]
def translate_sequence(input_seq, strand, rf):
    output_seq = ''
    if strand == 1:
        seq = input_seq[rf:]
    else:
        seq = align.reverse_complement(input_seq)[rf:]
    for i in range(0,len(seq)-2,3):
        if seq[i:(i+3)] in codon_dict:
            output_seq += codon_dict[seq[i:(i+3)]]
        else:
            output_seq += 'X'
    return output_seq

In [106]:
def translate_orf(sequence, start, end, strand):
    return translate_sequence(sequence[start:end], strand, 0)              

##### Output nt sequences in FASTA format

In [107]:
def produce_blast_file(record_list, output_filename):
    with open(output_filename, 'w',  newline='') as outfile:
        line_length = 60
        for record in tqdm(record_list):
            sequence = record[1]
            lines = []
            sequence_length = len(sequence)
            number_of_lines = math.ceil(sequence_length / line_length)
            lines.append(">" +record[0]+ "\n")
            for i in range(number_of_lines):
                subsequence = sequence[i*line_length:(i+1)*line_length]
                lines.append(subsequence + "\n")
            outfile.write(''.join(lines))

##### Function to find maximal open reading frame between two co-ordinates with mutation probability less than defined p-value

In [108]:
def find_all_maximal_orfs(sequence, seq_start, seq_stop, output_all_orfs = False, min_orf_length = 0):
    max_len = 0
    orfs_found = []
    start_pos = -999
    end_pos = -999
    for frame in ['Forward', 'Reverse']:
        if frame == 'Forward':
            temp = (sequence[seq_start: seq_stop])
        else:
            temp = align.reverse_complement(sequence[seq_start: seq_stop])
        seq_len = len(temp)
        for rf in range(3):
            i = rf
            while i < seq_len - 2:
                orf_length = 0
                test_codon = temp[i: i+3] 
                if test_codon in ['ATG','GTG','TTG']:  
                    for j in range(i + 3, seq_len - 2, 3):
                        test_codon_2 = temp[j: j+3] 
                        if test_codon_2 in ['TAG','TGA','TAA']:
                            orf_length = j - i
                            break
                            
                if orf_length > 0:
                    if frame == 'Forward':
                        orf_start =  seq_start + i
                        orf_end = seq_start + j+3
                        orf_strand = 1
                    else:
                        orf_start =  seq_start + seq_len-(j+3)
                        orf_end = seq_start + seq_len-i
                        orf_strand = -1
                    
                    if orf_length >= min_orf_length:
                        orfs_found.append((orf_start, orf_end, orf_strand, orf_length))

                if orf_length > max_len and orf_length >= min_orf_length:                                           
                    max_len = orf_length
                    start_pos = orf_start
                    end_pos = orf_end
                    strand = orf_strand 

                if orf_length > 0:
                    i = j
                else:
                    i +=3
    if output_all_orfs == True:
        sorted_orfs = sorted(orfs_found, key=lambda x: x[3], reverse=True)
        return sorted_orfs                
    elif start_pos == -999:
        return(0,0,0,0)
    else:
        return(start_pos, end_pos, strand, max_len)   

##### Function to find nearest upstream start to an alignment making it an open reading frame

In [178]:
def find_nearest_orf_sequence(sequence, seq_start, seq_stop, strand, max_lookback = 100):
    out_seq = ''
    if strand == 1:
            temp = (sequence[seq_start - max_lookback * 3: seq_stop])
    else:
            temp = align.reverse_complement(sequence[seq_start: seq_stop + max_lookback * 3])
    for lookback in range(max_lookback, -1, -1):
        if temp[lookback * 3: (lookback+1) *3 ] in ['ATG','GTG','TTG']:
            out_seq = translate_sequence(temp[lookback * 3:],1,0)
            break
    return(out_seq)   

##### Extract full sequences from each organism and create directory of start and stops for each annotated cds (use Mycobrowser for MTb)

In [115]:
def generate_myco_info(num_subsets, subset_num, species_master_list):
    output = []
    species_list = util.chunk_list(species_master_list, num_subsets, subset_num)
    for i, species in tqdm(enumerate(species_list)):
        features = []
        genome_record = next(SeqIO.parse(refseq_dir + '/'+species+'/genomic.gbff', "genbank"))
        full_sequence = str(genome_record.seq)
        if full_sequence.count('A') + full_sequence.count('C') + full_sequence.count('G') + full_sequence.count('T') < len(full_sequence):
            continue
        organism = genome_record.annotations['organism']
        
        #  Read feature information
        if species == reference_species:
            mycobrowser_df = pd.read_excel(literature_datasets_dir+'/Mycobrowser_Release_4.xlsx')
            for i, r in mycobrowser_df.iterrows():
                if r['Strand'] == '+':
                    strand = 1
                else:
                    strand = -1
                features.append((r['Locus'],r['Start']-1, r['Stop'], strand))
            
        else:
            
            for feature in genome_record.features:
                    a = feature.qualifiers
                    if a.get("locus_tag")!= None and int(feature.location.end) - int(feature.location.start) < 100000:  #  Exclude strange Biopython parsing where starts with complement join and looks like a CDS is full length of genome!    and feature.type == 'CDS':
                        locus_tag = a.get("locus_tag")[0]
                        features.append([locus_tag, int(feature.location.start), int(feature.location.end), int(feature.location.strand)])
        
        #  Find maximal orfs, non-overlapping orfs and their protein sequences, and assign a reference to each
        maximal_orfs = find_all_maximal_orfs(full_sequence, 0, len(full_sequence), True, min_nts)
        
        protein_references = []
        maximal_orf_proteins = []
        for i, orf in enumerate(maximal_orfs):
            temp = translate_orf(full_sequence, orf[0], orf[1], orf[2])
            maximal_orf_proteins.append([species+'_'+str(i), temp[:-1]])
            protein_references.append((species+'_'+str(i), orf))

        non_overlapping_maximal_orfs = []
        for orf in maximal_orfs:
            overlap = False
            for cds in features:
                if min(cds[2], orf[1]) - max(cds[1], orf[0]) > 0.3 * (orf[1] - orf[0]):
                    overlap = True
                    break
            if overlap == False:
                non_overlapping_maximal_orfs.append(orf)
                
        non_overlapping_maximal_orf_proteins = []
        for i, orf in enumerate(non_overlapping_maximal_orfs):
            temp = translate_orf(full_sequence, orf[0], orf[1], orf[2])
            non_overlapping_maximal_orf_proteins.append([species+'_NO_'+str(i), temp[:-1]])
            protein_references.append((species+'_NO_'+str(i), orf))
        
            
        
        output.append((species, organism, full_sequence, features, maximal_orfs, non_overlapping_maximal_orfs, maximal_orf_proteins, non_overlapping_maximal_orf_proteins, protein_references))
    return output

In [116]:
myco_info_dict = {}
names_dict = {}
parallel_output = Parallel(n_jobs=-1)(delayed(generate_myco_info)(num_cores, core_number, species_list) for core_number in core_numbers)
for core_output in parallel_output:
    for results in core_output:
        myco_info_dict[results[0]] = (results[1], results[2], results[3], results[4], results[5], results[6], results[7], results[8])
        names_dict[results[0]] = results[1]

In [120]:
protein_reference_dict = {}
for species in species_list:
    for ref in myco_info_dict[species][7]:
        protein_reference_dict[ref[0]] = ref[1]

##### Output two blast files - one (subject) containing all translated mORFs, the other (query) just the ones for the reference species 

In [121]:
subject_protein_list = []
for species in species_list:
    for maximal_orf_protein in myco_info_dict[species][5]:
        subject_protein_list.append(maximal_orf_protein)
produce_blast_file(subject_protein_list, refseq_dir + '/subject_proteins.faa')

query_protein_list = myco_info_dict[reference_species][6]
produce_blast_file(query_protein_list, refseq_dir + '/no_overlap_morf_query_proteins.faa')

query_protein_list = myco_info_dict[reference_species][5]
produce_blast_file(query_protein_list, refseq_dir + '/morf_query_proteins.faa')

  0%|          | 0/5881544 [00:00<?, ?it/s]

  0%|          | 0/4377 [00:00<?, ?it/s]

  0%|          | 0/59882 [00:00<?, ?it/s]

##### Create blast database

In [122]:
if 1==1:
    w_d = os.getcwd()
    os.chdir("D:/")
    subprocess.run('cd d:\\Tests\\NCBI_Dataset_Mycobacteria & makeblastdb -in subject_proteins.faa -dbtype prot -out max_prot', shell=True, capture_output = True)
    os.chdir(w_d)

In [123]:
if 1==1:
    w_d = os.getcwd()
    os.chdir("D:/")
    subprocess.run('cd d:\\BLAST\\maxprot & blastp -query d:\\Tests\\NCBI_Dataset_Mycobacteria\\morf_query_proteins.faa -db max_prot -out blastp_results2.csv -evalue 1e-6 -seg no -outfmt  "10 qaccver saccver qlen slen pident length mismatch gapopen qstart qend sstart send sstrand evalue bitscore" -num_threads 16', shell=True, capture_output = True)
    os.chdir(w_d)

In [124]:
blast_results = pd.read_csv("D:\\BLAST\\maxprot\\blastp_results2.csv", header = None)
blast_results.columns = ['query_accession_ver', 'subject_accession_ver', 'query_length', 'subject_length', 'percent_identical_matches','alignment_length', 'number_mismatches', 'number_of_gap_openings', 'query_start_alignment', 'query_end_alignment', 'subject_start_alignment', 'subject_end_alignment', 'subject_strand', 'e_value', 'bit_score']

In [125]:
blast_results = blast_results.query('subject_length == subject_end_alignment and query_length == query_end_alignment')
blast_results['query_info']=  blast_results['query_accession_ver'].map(protein_reference_dict)
blast_results['subject_info']=  blast_results['subject_accession_ver'].map(protein_reference_dict)

In [126]:
for i, r in blast_results.iterrows():
    blast_results.at[i, 'query_species'] = '_'.join(r.query_accession_ver.split('_')[0:2])
    blast_results.at[i, 'subject_species'] = '_'.join(r.subject_accession_ver.split('_')[0:2])
blast_results = blast_results.query('not (query_species == subject_species)')
blast_results['query_species_name'] = blast_results['query_species'].map(names_dict)
blast_results['subject_species_name'] = blast_results['subject_species'].map(names_dict)

In [127]:
for i, r in blast_results.iterrows():
    if r.query_info[2] == 1:
        blast_results.at[i, 'query_start_pos'] = r.query_info[0] + (r.query_start_alignment - 1) * 3
        blast_results.at[i, 'query_end_pos'] = r.query_info[0] + (r.query_end_alignment - 1) * 3 + 6 # doesn't include stop codon in blast
    else:
        blast_results.at[i, 'query_start_pos'] = r.query_info[0] + (r.query_end_alignment - r.query_length) * 3
        blast_results.at[i, 'query_end_pos'] = r.query_info[1] - (r.query_start_alignment - 1) * 3
       
    if r.subject_info[2] == 1:
        blast_results.at[i, 'subject_start_pos'] = r.subject_info[0] + (r.subject_start_alignment - 1) * 3
        blast_results.at[i, 'subject_end_pos'] = r.subject_info[0] + (r.subject_end_alignment - 1) * 3 + 6 # doesn't include stop codon in blast
    else:
        blast_results.at[i, 'subject_start_pos'] = r.subject_info[0] + (r.subject_end_alignment - r.subject_length) * 3
        blast_results.at[i, 'subject_end_pos'] = r.subject_info[1] - (r.subject_start_alignment - 1) * 3

In [159]:
blast_results = blast_results.loc[blast_results.groupby(['query_accession_ver','subject_species'])['bit_score'].idxmax()]
blast_results['species_count'] = blast_results.groupby('query_accession_ver')['query_accession_ver'].transform('size')

In [160]:
blast_results.to_csv(project_dir + '/blast_results_tb_maximal_orfs.csv')

In [183]:
df = blast_results[blast_results.species_count > 7]
for i, r in df.iterrows():
    subject_strand = r.subject_info[2]
    df.at[i,'sequence'] = translate_orf(myco_info_dict[r.subject_species][1], int(r.subject_start_pos), int(r.subject_end_pos), subject_strand)
    df.at[i,'nearest_upstream_orf_sequence'] = find_nearest_orf_sequence(myco_info_dict[r.subject_species][1], int(r.subject_start_pos), int(r.subject_end_pos), subject_strand, max_lookback = 100)

C:\Users\nicho\AppData\Local\Temp/ipykernel_16392/1169293613.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[i,'sequence'] = translate_orf(myco_info_dict[r.subject_species][1], int(r.subject_start_pos), int(r.subject_end_pos), subject_strand)
C:\Users\nicho\AppData\Local\Temp/ipykernel_16392/1169293613.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[i,'nearest_upstream_orf_sequence'] = find_nearest_orf_sequence(myco_info_dict[r.subject_species][1], int(r.subject_start_pos), int(r.subject

In [184]:
df.to_csv(project_dir + '/blast_results_maximal_orfs.high_hits.csv')

In [ ]:
tb_blast_results = blast_results[blast_results.query_species == reference_species]
plt.figure(figsize=(16, 16))
g = sns.FacetGrid(tb_blast_results, col='subject_species', height=6, col_wrap=3)
g.map(sns.scatterplot, 'subject_start_pos', 'query_start_pos', s=20)
#sns.scatterplot(data = blast_results, y = 'subject_start_pos', x = 'query_start_pos', s=1, hue = 'annotated_cds_in_query')

##### Load variant dataset and create variant dictionary (0 indexed for genome position - whereas CRyPtiC data uses 1 start)

In [147]:
variant_count_df = pd.read_csv(cryptic_output_path + '/filtered_variant_summary_df.csv')
variant_count_df = variant_count_df[variant_count_df['MUTATION_PCT'] < 0.1]

In [148]:
temp = variant_count_df.groupby(['GENOME_INDEX'])[['MYKROBE_LINEAGE_NAME_2']].count().reset_index()
temp_dict = dict(zip(temp.GENOME_INDEX, temp.MYKROBE_LINEAGE_NAME_2))

In [150]:
mutation_counts_dict = {}
for i in tqdm(range(len(myco_info_dict[reference_species][1]))):
    if (i+1) in temp_dict:
        mutation_counts_dict[i] = temp_dict[(i+1)]
    else:
        mutation_counts_dict[i] = 0

  0%|          | 0/4411532 [00:00<?, ?it/s]

##### Define binomial probabilities for testing mutation counts

In [151]:
def bin_formula(max_bin_counts, tot_bin_counts, in_frame = False):
    return 1- binom.cdf(max_bin_counts-1, tot_bin_counts,1/3)

In [152]:
def mutation_bin_probability(start, end, strand):
    mutations = []
    for i in range(start,end):
        for j in range(mutation_counts_dict[i]):
            mutations.append(i)
    bin_counts = [0,0,0]
    for m in mutations:
        if strand == 1:
            bin_counts[(m-(start))%3] +=1
        else:
            bin_counts[((end-1)-m)%3] +=1
    if sum(bin_counts) == 0:
        return (2)
    else:
        return (bin_formula(bin_counts[2], sum(bin_counts)))  

In [153]:
mutation_bin_probability(1282029, 1282218,1)

0.17660632866964088

In [ ]:
for orf in non_overlapping_query_maximal_orfs:
    temp = mutation_bin_probability(orf[0],orf[1],orf[2]) 
    if temp < 0.0002:
        print(orf, mutation_bin_probability(orf[0],orf[1],orf[2]))

In [ ]:
species_organism_dict

In [ ]:
orfs_maximal_in_gaps = []
non_included_boundaries = []
for i, (locus,start,stop,strand) in enumerate(cds_boundary_dict[reference_species]):
    if i==0 or stop >= cds_boundary_dict[reference_species][i-1][2]:
        non_included_boundaries.append(cds_boundary_dict[reference_species][i])

for i, (locus,start,stop,strand) in enumerate(non_included_boundaries):
    temp = []
    if i < len(non_included_boundaries) - 1:
        temp = find_all_maximal_orfs(full_sequences[reference_species_index], stop, non_included_boundaries[i+1][1], output_all_orfs = True, min_orf_length = 30)
        for orf in temp:
            orfs_maximal_in_gaps.append(orf)
            